# Práctica 2 Parte 1

La primera parte consiste en probar el método implementado en la práctica 1 con diferentes experimentos, en este caso lo probaremos con distintas redes y hacerlo más robusto. Los ejemplos auxiliares serán utilizados para encontrar errores o áreas de oportunidad de nuestro algoritmo.

Para consultar la información de nuestro paquete, vea: [reporte parte 2 práctica 1](https://github.com/optimizacion-2-2022-gh-classroom/practica-1-segunda-parte-LuzVerde23/blob/main/reporte_equipo_2_parte_2_practica_1.ipynb).


In [2]:
import pandas as pd
import numpy as np
#import kale
from MaxFlowAeiu import MaxFlowAeiu
import time

## Ejemplo 1
Para comenzar usaremos el ejemplo de la práctica uno

In [8]:
url_d = "https://raw.githubusercontent.com/optimizacion-2-2022-gh-classroom/practica-2-primera-parte-urieluard/main/BD/d.csv"
#https://github.com/optimizacion-2-2022-gh-classroom/practica-2-primera-parte-urieluard/blob/main/BD/d.csv

In [9]:
d = pd.read_csv(url_d,header=None)

In [10]:
d.head()

,0,1,2,3,4,5,6,7,8,9,...,34,35,36,37,38,39,40,41,42,43
0,0.0,2000.0,0.0,2000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,535.0,265.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,600.0,0.0,1200.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1600.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Generamos el arreglo final de tipo "numpy array"
arreglo = d.to_numpy()

In [12]:
%%time
MF=MaxFlowAeiu(arreglo)

CPU times: user 17 µs, sys: 0 ns, total: 17 µs
Wall time: 21.5 µs


In [13]:
%%time
print("The maximum flow in this network is {}".format(MF.ford_fulkerson()))

The maximum flow in this network is 1480.0
CPU times: user 3.85 ms, sys: 803 µs, total: 4.65 ms
Wall time: 7.68 ms


# Ejemplo 2
Seguimos con un ejemplo de menor escala que trata sobre el transporte de personas en una empresa de autobúses

In [6]:
d2 = [[0,188,240,160,0,0,0,0,0,0,0,0,0],
                    [0,0,0,0,170,0,0,0,0,0,0,0,0],
                    [0,0,0,0,0,0,210,180,0,0,0,0,0],
                    [0,0,0,0,0,140,0,0,0,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0,130,90,0,0],
                    [0,0,0,0,0,0,0,0,130,0,0,0,0],
                    [0,0,0,0,0,0,0,0,150,0,0,0,0],
                    [0,0,0,0,0,0,0,0,0,0,0,160,0],
                    [0,0,0,0,0,0,0,0,0,0,0,0,210],
                    [0,0,0,0,0,0,0,0,0,0,0,0,110],
                    [0,0,0,0,0,0,0,0,0,0,0,0,80],
                    [0,0,0,0,0,0,0,0,0,0,0,0,170],
                    [0,0,0,0,0,0,0,0,0,0,0,0,0]]


In [7]:
# Generamos el arreglo final de tipo "numpy array"
arreglo2 = np.array(d2)

In [8]:
%%time
MF2=MaxFlowAeiu(arreglo2)

CPU times: user 17 µs, sys: 3 µs, total: 20 µs
Wall time: 27.2 µs


In [9]:
%%time
print("The maximum flow in this network is {}".format(MF2.ford_fulkerson()))

The maximum flow in this network is 540
CPU times: user 3.32 ms, sys: 52 µs, total: 3.37 ms
Wall time: 3.15 ms


## Ejemplo 3
Ahora probaremos con una matriz más grande sobre el transporte de pasajeros en aerolíneas en Estados Unidos

In [10]:
d3=pd.read_csv("https://raw.githubusercontent.com/sid-7/Airline_Maximum_Flow/master/flights.csv")
d3=d3.drop(['Unnamed: 0', 'Depature', 'Arrival'], axis=1)
d3=d3.groupby(['Source', 'Destination'], as_index=False)['capacity'].sum()
d3['capacity'] = pd.to_numeric(d3['capacity'], errors='coerce')
new_row = pd.DataFrame([['JFK','JFK', 0], ['LAX','LAX', 0]],
                   columns=['Source', 'Destination', 'capacity'])
d3 = pd.concat([d3, new_row])
d3 = d3.pivot(index="Source", columns="Destination", values="capacity").fillna(0)
column_to_move1 = d3.pop("LAX")
column_to_move2 = d3.pop("JFK")
d3.insert(0, "LAX", column_to_move1)
d3.insert(9, "JFK", column_to_move2)
t1 = ['JFK','LAX']
t2 = ['JFK']
t3 = ['LAX']

a = d3.loc[[i for i in d3.index if i not in t1], :]
b = d3.loc[t2, :]
c = d3.loc[t3, :]

d3=pd.concat([c, a, b])
d3.head(10)

Destination,LAX,ATL,BOS,DEN,IAD,ORD,PHX,SEA,SFO,JFK
Source,,,,,,,,,,
LAX,0.0,2288.0,836.0,2072.0,520.0,2525.0,1156.0,1624.0,3842.0,2057.0
ATL,0.0,0.0,2035.0,1440.0,884.0,2235.0,1573.0,1180.0,1618.0,1350.0
BOS,0.0,1850.0,0.0,720.0,720.0,1884.0,640.0,180.0,1380.0,751.0
DEN,0.0,1688.0,720.0,0.0,1080.0,2150.0,1372.0,1379.0,1790.0,360.0
IAD,0.0,938.0,638.0,1180.0,0.0,938.0,180.0,488.0,1130.0,0.0
ORD,0.0,2500.0,2185.0,2660.0,728.0,0.0,2148.0,1584.0,2437.0,160.0
PHX,0.0,1270.0,480.0,1352.0,180.0,2022.0,0.0,1194.0,1431.0,1187.0
SEA,0.0,900.0,165.0,1022.0,308.0,1404.0,1108.0,0.0,2047.0,460.0
SFO,0.0,1508.0,920.0,1690.0,760.0,1930.0,1254.0,2290.0,0.0,102.0


In [11]:
# Generamos el arreglo final de tipo "numpy array"
arreglo3 = np.array(d3)

In [12]:
%%time
MF3=MaxFlowAeiu(arreglo3)

CPU times: user 24 µs, sys: 0 ns, total: 24 µs
Wall time: 33.6 µs


In [13]:
%%time
print("The maximum flow in this network is {}".format(MF3.ford_fulkerson()))

The maximum flow in this network is 6427.0
CPU times: user 6.44 ms, sys: 1.07 ms, total: 7.51 ms
Wall time: 5.62 ms


## Ejemplo 4
Ahora usaremos una matriz mucho más grande

In [14]:
import networkx as nx
import random

In [15]:
# Especificar el tamaño de la red n = número de nodos, m = número de ramas
n = 1000
m = 5000

In [16]:
network = nx.gnm_random_graph(n, m, directed=True)
nodos = list(network.nodes)
ramas = list(network.edges)

In [17]:
# Crear arreglos de: nodo de envío, nodo de recepción y capacidad del enlace
env = []
rec = []
cap = []
for i in range(len(ramas)):
    env.append(ramas[i][0])
    rec.append(ramas[i][1])
    cap.append(random.randint(20,100)) # La capacidad del enlace se determinade forma aleatoria con valores entre 20 y 100

In [18]:
# Se crea el data frame o matriz de incidencias de los nodos. Se asegura que el nodo fuente sea el 0 y solo tenga ramas saliendo de él y que el último nodo solo tenga ramas entrando (sumidero)
d4 = pd.DataFrame(0, index=np.arange(0,len(nodos),1), columns=list(range(0,len(nodos))))
aux = 0
for n in env:
    if rec[aux] == 0:
        d4.iloc[rec[aux],n] = cap[aux]
    elif n == max(nodos):
        d4.iloc[rec[aux],n] = cap[aux]
    else:
        d4.iloc[n,rec[aux]] = cap[aux]
    aux += 1

d4

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,27,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# Generamos el arreglo final de tipo "numpy array"
arreglo4 = d4.to_numpy()

In [20]:
%%time
MF4=MaxFlowAeiu(arreglo4)

CPU times: user 24 µs, sys: 2 µs, total: 26 µs
Wall time: 35 µs


In [21]:
%%time
print("The maximum flow in this network is {}".format(MF4.ford_fulkerson()))

The maximum flow in this network is 643
CPU times: user 26 s, sys: 0 ns, total: 26 s
Wall time: 26 s


## Conclusiones

Ya vimos que el desempeño de nuestro algoritmo da buenos resultados en cuanto a tiempo en los primeros 3 ejemplos fue casi imperceptible, para el ejemplo con una red de 1000 nodos con 5000 arcos tomó 26 segundos en obtener el flujo máximo, lo cual sigue siendo bueno.

## Referencias
* [1] [Palacios E. (2022) Libro de Optimización](https://itam-ds.github.io/analisis-numerico-computo-cientifico/4.optimizacion_en_redes_y_prog_lineal/4.2/Definiciones_generales_de_flujo_en_redes.html)
* [2] [Dumora c. el all. Data Oriented Algorithm for Real Time Estimation of Flow Rates and Flow Directions in Water Distribution Network](https://arxiv.org/pdf/1807.10147.pdf)
* [3] [Max Flow Problem Introduction](https://www.geeksforgeeks.org/max-flow-problem-introduction/)
* [4] [Ford-Fulkerson Algorithm](https://www.programiz.com/dsa/ford-fulkerson-algorithm)
* [5] [Algoritmo de Ford-Fulkerson - Ford–Fulkerson algorithm](https://upwikies.top/wiki/Ford%e2%80%93Fulkerson_algorithm)
* [6] [Oviedo J. (2008) Algoritmo de Ford-Fulkerson Mejorado](http://www.ptolomeo.unam.mx:8080/jspui/bitstream/132.248.52.100/2387/1/gonzalezoviedo.pdf)
* [7] [Building a Smarter (and Cheaper) School Bus System: How a Boston-MIT Partnership Led to New Routes That Are 20% More Efficient and Saved the District $5 Million](https://www.the74million.org/article/building-a-smarter-and-cheaper-school-bus-system-how-a-boston-mit-partnership-led-to-new-routes-that-are-20-more-efficient-use-400-fewer-buses-save-5-million/)
* [8] [Optimazation examples](https://vitalflux.com/convex-optimization-explained-concepts-examples/)
* [9] [Breadth First Search or BFS for a Graph](https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/)